In [3]:
from graphframes import *
from functools import reduce
from pyspark import *
from pyspark.sql import *
from operator import add
import shortuuid

In [ ]:
#with open("com-orkut.top5000.cmty.txt", 'r') as file:
with open("testgraph", 'r') as file:
    lines = file.read().split("\n")

print("lines")
#print(lines)
vertices = []
existing_vertices = set()
edges = []
hyperedges = {}
counter=0
for l in lines:
    if l == "":
        continue
    counter+=1
    all_line_vertices = l.split(" ")
    hypid = "he" + str(counter)
    try:
        all_line_vertices.remove("")
    except:
        pass
    hyperedges[hypid] = all_line_vertices
    for v in all_line_vertices:
        if not v in existing_vertices:
            vertices.append((v,"vertex"))
            existing_vertices.add(v)
        if not hypid in existing_vertices:
            vertices.append((hypid,"hyperedge"))
            existing_vertices.add(hypid)
        edges.append((v,hypid))
        edges.append((hypid,v))

vertex_metadata = ['id','type']
edges_metadata = ['src','dst']
#print(vertices)
#print(edges)

vertices = spark.createDataFrame(vertices,vertex_metadata)
edges = spark.createDataFrame(edges,edges_metadata)



print("EDGESRCDEST")
print(hyperedges)
hyp_deg = spark.sparkContext.broadcast(hyperedges)
g = GraphFrame(vertices, edges)## Take a look at the DataFrames

In [32]:
from pyspark.sql.types import Row
import uuid
from pyspark.sql.functions import monotonically_increasing_id
from uuid import UUID

def f(x, uuid):
    #return 1,x.value.split(" ")[0]
    d = {}
    elements = x.value.split(" ")
    for i in range(len(elements)):
        yield elements[i],uuid
        yield uuid, elements[i]
    #for i in range(len(x)):
    #    d["abc"] = x[i]
    #return d

def make_edge_df(edge):
    d = {}
    d["src"] = edge[0]
    d["dst"] = edge[1]
    return d

def make_vertex_df(edge):
    yield edge.src,"vertex"
    yield edge.dst, "vertex"
#     try:
#         uuid_obj = UUID(edge[0])
#         yield edge[0],"hyperedge"
#     except ValueError:
#         yield edge[0], "vertex"
    
#     try:
#         uuid_obj = UUID(edge[1])
#         yield edge[1],"hyperedge"
#     except ValueError:
#         yield edge[1], "vertex"
    
        
def hyp_df(x, delimit):
    return x.value.split(delimit)

def parse_hyp(x):
    hypid = "hyp" + str(x[len(x)-1])
    for i in range(len(x)-1):
        yield x[i],hypid
        yield hypid,x[i]

#Now populate that
#lines = spark.read.text("testgraph").rdd.flatMap(lambda x: f(x,str(uuid.uuid4())))#.toDF()

hyp_lines = spark.read.text("testgraph").rdd.map(lambda x: hyp_df(x, " ")).toDF()
hyp_lines = hyp_lines.withColumn("id", monotonically_increasing_id())

df_vert = hyp_lines.rdd.flatMap(lambda x: parse_hyp(x)).toDF(["src","dst"])
print("df col",df_vert.show())

#new_test = [list(row) for row in hyp_lines.collect()]
#print(new_test)
hyp_lines.show()


#df_edges = lines.map(lambda x:Row(**make_edge_df(x))).toDF()

#df_edges = lines.map(lambda x:(x[0],x[1])).toDF(["src","dst"])

#df_vertices = df_edges.rdd.map(lambda x:(x.src,x.dst)).toDF(["id","type"])
#print(df_edges.show())
#df_vertices.show()
#print(lines.collect())
#df = rdd.map(lambda x: Row(**f(x))).toDF()
    

+----+----+
| src| dst|
+----+----+
|   0|hyp0|
|hyp0|   0|
|   1|hyp0|
|hyp0|   1|
+----+----+

df col None
+---+---+---+
| _1| _2| id|
+---+---+---+
|  0|  1|  0|
+---+---+---+



In [ ]:
df_vertices.filter('id == "56d21a1f-4342-4d22-a486-3ae4960c9890"').show()

In [ ]:
vertex_metadata = ['id','type']
edges_metadata = ['src','dst']
vertices = []
existing_vertices = set()
vex = []
hyperedges = {}
counter=0
existing_vertices = set()
with open("com-orkut.top5000.cmty.txt", 'r') as file:
    lines = file.read().split("\n")

for l in lines:
    if l == "":
        continue
    counter+=1
    all_line_vertices = l.split("\t")
    hypid = "he" + str(counter)
    try:
        all_line_vertices.remove("")
    except:
        pass
    hyperedges[hypid] = all_line_vertices
    for v in all_line_vertices:
        if not v in existing_vertices:
            try:
                frame_vertices = frame_vertices.union(spark.createDataFrame([(v,"vertex")],vertex_metadata))
            except:
                frame_vertices = spark.createDataFrame([(v,"vertex")],vertex_metadata)
            vex.append((v,"vertex"))
            existing_vertices.add(v)
        if not hypid in existing_vertices:
            try:
                frame_vertices = frame_vertices.union(spark.createDataFrame([(hypid,"hyperedge")],vertex_metadata))
            except:
                frame_vertices = spark.createDataFrame([(hypid,"hyperedge")],vertex_metadata)
            vex.append((hypid,"hyperedge"))
            existing_vertices.add(hypid)
        if "ev" in locals() or "ev" in globals():
            ev = ev.union(spark.createDataFrame([(v,hypid)],edges_metadata))
            ev = ev.union(spark.createDataFrame([(hypid,v)],edges_metadata))
        else:
            ev = spark.createDataFrame([(v,hypid)],edges_metadata)
            ev = evg.union(spark.createDataFrame([(hypid,v)],edges_metadata))
#         edges.append((v,hypid))
#         edges.append((hypid,v))

edges.show()
frame_vertices.show()
print(vex)

In [ ]:
g.edges.show()
g.vertices.show()

In [ ]:

def initialize_page_rank(vertex, count):
    return 1/count

def parseNeighbors(edge):
    dest_id = edge.dst
    if edge.dst in hyperedges:
        return edge.src, hyperedges[edge.dst]
    return edge.src,[]

In [ ]:
def getContributions(ur):
    len_urls = len(ur[1][0])
    for site in sum(ur[1][0],[]):
        yield site, ur[1][1]/(len_urls-1)
    

In [ ]:
def parallel_pagerank(iterations, graph):
    all_vertices =g.vertices.filter('type=="vertex"')
    num_vertices = all_vertices.count()
    all_edges = graph.edges
    ranks = all_vertices.rdd.map(lambda x:(x.id,initialize_page_rank(x, num_vertices))) #
    hyperedge_links = all_edges.rdd.map(lambda el: parseNeighbors(el)).groupByKey().cache() # vertex hyp1 hyp2 hyp3
    for i in range(iterations):
        result_links = hyperedge_links.join(ranks).flatMap(lambda ur:getContributions(ur))
        ranks = result_links.reduceByKey(add).mapValues(lambda rank: rank * 0.15 + 0.15)
    print(ranks.collect())

In [ ]:
parallel_pagerank(5,g)

In [ ]:
g.edges.filter('src == "1"').show()

In [ ]:
def get_outgoing(node_id, graph):
    df2 = g.edges.filter('src=="'+node_id+'"')
    df2.show()
    df1 = graph.edges
    df3 = df1.alias("df1").join(broadcast(df2.alias("df2")), col("df2.dst") == col("df1.src")).select(df1["dst"])
    df3.distinct().show() 

def get_incoming(node_id, graph):
    df2 = g.edges.filter('dst=="'+node_id+'"')
    df2.show()
    df1 = graph.edges
    df3 = df1.alias("df1").join(broadcast(df2.alias("df2")), col("df2.src") == col("df1.dst")).select(df1["src"])
    df3.distinct().show() 